# TASK 1

In [6]:
class Polynomial:
    def __init__(self, *args):
        if len(args) == 0:
            self.coefs = 0
        else:
            self.coefs = [coef for coef in args]
        
    def __call__(self, x):
        if self.coefs == 0:
            return 0
        return sum([x ** idx * coef for idx, coef in enumerate(self.coefs)])

In [7]:
my_polynom = Polynomial(2, 3, 1)

In [8]:
print(my_polynom(4))

30


In [9]:
for i, f in enumerate([2, 3, 4]):
    print(f)

2
3
4


# TASK 2, 3, 4

In [2]:
class CooSparseMatrix:
    def __init__(self, ijx_list, shape):
        self.ijx_list = ijx_list
        self.ijx_dict = {}
        self.shape = shape
        for ijx in self.ijx_list:
            ij, x = ijx[:2], ijx[2]
            is_i_int = isinstance(ij[0], int)
            is_j_int = isinstance(ij[1], int)
            if (not is_i_int) | (not is_j_int):
                raise TypeError
            if ij in self.ijx_dict.keys():
                raise TypeError
            else:
                self.ijx_dict[ij] = x
    
    def __setattr__(self, name, value):                
        if name == 'shape':
            if 'shape' not in self.__dict__:
                self.__dict__[name] = value
            else:
                elems_amount = self.shape[0] * self.shape[1]
                if not isinstance(value, tuple):
                    raise TypeError
                elif (value[0] <= 0) | (value[1] <= 0):
                    raise TypeError
                elif (value[0] * value[1] != elems_amount):
                    raise TypeError
                else:
                    ij_sorted = sorted(self.ijx_dict.keys())
                    position_in_1d_2d = \
                        {i * self.shape[1] + j :
                         (i, j) for i, j in self.ijx_dict.keys()}
                    ijx_dict_new = {}
                    for val in position_in_1d_2d.keys():
                        new_i = int(val / value[1])
                        new_j = val - value[1] * new_i
                        ijx_dict_new[(new_i, new_j)] = self.ijx_dict[position_in_1d_2d[val]]
                    self.ijx_dict = ijx_dict_new
                    self.__dict__[name] = value 
        elif name == 'T':
            raise AttributeError
        else: 
            self.__dict__[name] = value
    
    def __getattr__(self, item):
        if item == 'T':
            ijx_list_new = [(ijx[1], ijx[0], self.ijx_dict[ijx]) for ijx in self.ijx_dict.keys()]
            shape_new = self.shape[1], self.shape[0]
            sparse_transposed = CooSparseMatrix(ijx_list_new, shape_new)
            return sparse_transposed
        else:
            return self.__dict__[item]
            
    def __getitem__(self, idx):
        if isinstance(idx, int):
            ijx_list_new = []
            if (idx >= self.shape[0]) | (idx < 0):
                raise TypeError
            for ij in self.ijx_dict.keys():
                if idx == ij[0]:
                    x = self.ijx_dict[ij]
                    ijx_list_new.append((0, ij[1], x))
            shape_new = (1, self.shape[1])
            sparse_new = CooSparseMatrix(ijx_list_new, shape_new)
            return sparse_new
        else:
            row_amount = self.shape[0]
            col_amount = self.shape[1]
            is_idx = (idx[0] in range(row_amount)) & \
                     (idx[1] in range(col_amount))
            if idx in self.ijx_dict.keys():
                return self.ijx_dict[idx]
            elif is_idx:
                return 0
            else:
                raise TypeError
    
    def __setitem__(self, ij, x):
        row_amount = self.shape[0]
        col_amount = self.shape[1]
        is_idx = (ij[0] in range(row_amount)) & \
                 (ij[1] in range(col_amount))
        if (x == 0) & (ij in self.ijx_dict.keys()):
            self.ijx_dict.pop(ij)
        if (is_idx) & (x != 0):
            self.ijx_dict[ij] = x
        elif not is_idx:
            raise KeyError(ij)

    def __add__(self, other):
        if self.shape != other.shape:
            raise TypeError
        if len(self.ijx_dict.keys()) == 0:
            return other
        if len(other.ijx_dict.keys()) == 0:
            return self
        ij_wout_zero = other.ijx_dict.keys() | self.ijx_dict.keys()
        sparse_new = CooSparseMatrix([], self.shape)  
        for ij in ij_wout_zero:
            sparse_new[ij] = self[ij] + other[ij]
        return sparse_new
    
    def __sub__(self, other):
        if self.shape != other.shape:
            raise TypeError
        if (self.ijx_dict == other.ijx_dict):
            return CooSparseMatrix([], self.shape)
        if len(other.ijx_dict.keys()) == 0:
            return self
        ij_wout_zero = other.ijx_dict.keys() | self.ijx_dict.keys()
        sparse_new = CooSparseMatrix([], self.shape)
        for ij in ij_wout_zero:
            if self[ij] != other[ij]:
                sparse_new[ij] = self[ij] - other[ij]
        return sparse_new
    
    def __mul__(self, other):
        sparse_new = CooSparseMatrix([], self.shape)
        if other == 0:
            return sparse_new
        for ij in self.ijx_dict.keys():
            sparse_new[ij] = self[ij] * other
        return sparse_new
    
    def __rmul__(self, other):
        sparse_new = CooSparseMatrix([], self.shape)
        if other == 0:
            return sparse_new
        for ij in self.ijx_dict.keys():
            sparse_new[ij] = self[ij] * other
        return sparse_new

In [1]:
matrix = CooSparseMatrix(ijx_list=[(0, 0, 1), (1, 1, 2), (2, 3, 5), (1, 3, 0)], shape=(3, 5))

NameError: name 'CooSparseMatrix' is not defined

In [1]:
from sparse_matrix import CooSparseMatrix

matrix1 = CooSparseMatrix([], shape=(100, 100))

for i in range(100):
    for j in range(100):
        if i % 13 == 0:
            matrix1[i, j] = i - 2 * (j ** 2)
        if j % 11 == 0:
            matrix1[i, j] = j + 3 * (i ** 3)

matrix2 = matrix1.T
for i in range(3):
    for j in range(30):
        print(i, j, matrix2[i, j])
        print(i, j, matrix1[i, j])

0 0 0
0 0 0
0 1 3
0 1 -2
0 2 24
0 2 -8
0 3 81
0 3 -18
0 4 192
0 4 -32
0 5 375
0 5 -50
0 6 648
0 6 -72
0 7 1029
0 7 -98
0 8 1536
0 8 -128
0 9 2187
0 9 -162
0 10 3000
0 10 -200
0 11 3993
0 11 11
0 12 5184
0 12 -288
0 13 6591
0 13 -338
0 14 8232
0 14 -392
0 15 10125
0 15 -450
0 16 12288
0 16 -512
0 17 14739
0 17 -578
0 18 17496
0 18 -648
0 19 20577
0 19 -722
0 20 24000
0 20 -800
0 21 27783
0 21 -882
0 22 31944
0 22 22
0 23 36501
0 23 -1058
0 24 41472
0 24 -1152
0 25 46875
0 25 -1250
0 26 52728
0 26 -1352
0 27 59049
0 27 -1458
0 28 65856
0 28 -1568
0 29 73167
0 29 -1682
1 0 -2
1 0 3
1 1 0
1 1 0
1 2 0
1 2 0
1 3 0
1 3 0
1 4 0
1 4 0
1 5 0
1 5 0
1 6 0
1 6 0
1 7 0
1 7 0
1 8 0
1 8 0
1 9 0
1 9 0
1 10 0
1 10 0
1 11 0
1 11 14
1 12 0
1 12 0
1 13 11
1 13 0
1 14 0
1 14 0
1 15 0
1 15 0
1 16 0
1 16 0
1 17 0
1 17 0
1 18 0
1 18 0
1 19 0
1 19 0
1 20 0
1 20 0
1 21 0
1 21 0
1 22 0
1 22 25
1 23 0
1 23 0
1 24 0
1 24 0
1 25 0
1 25 0
1 26 24
1 26 0
1 27 0
1 27 0
1 28 0
1 28 0
1 29 0
1 29 0
2 0 -8
2 0 24
2 1 0
2 

In [6]:
matrix1.T.ijx_list

[]

In [66]:
matrix.T = 0

AttributeError: 

In [3]:
import numpy as np


def to_array(matrix):
    array = np.zeros(matrix.shape)
    for ij in matrix.ijx_dict.keys():
        array[ij] = matrix[ij]
    return array

In [6]:
to_array(matrix3)

array([[1., 2.],
       [3., 0.]])

In [74]:
{1:2, 2:3}.keys() & {1:2, 3:4}.keys()

{1}

In [26]:
sorted({(0, 1) :2, (1, 0):3, (1, 50):3, (0, 3):3, (0, 2):3}.keys())

[(0, 1), (0, 2), (0, 3), (1, 0), (1, 50)]

In [29]:
{i*3 + j : (i, j) for i in range(3) for j in range(3)}

{0: (0, 0),
 1: (0, 1),
 2: (0, 2),
 3: (1, 0),
 4: (1, 1),
 5: (1, 2),
 6: (2, 0),
 7: (2, 1),
 8: (2, 2)}

In [39]:
ik = int(7/3)

In [40]:
ik

2

In [41]:
jk = 7 - 3 * ik

In [42]:
jk

1